In [ ]:
# CELL 0 – Imports and physical constants

import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u   # ← THIS is what was missing
from scipy.interpolate import griddata

# Physical constants
G   = 6.67430e-11         # m^3 kg^-1 s^-2
c   = 2.99792458e8        # m/s
M_sun = 1.98847e30        # kg
kpc = 3.085677581e19      # m

# (optional) cosmology object if you need it later
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
# CELL 1 — Lens configuration input block (RX J1131–1231)
# =======================================================
# Change ONLY this cell for each lens.

# 1. Redshifts (lens & source)
z_d = 0.295     # lens redshift
z_s = 0.658     # source redshift

# 2. Angular-diameter distances (use the cosmology from CELL 0)
#    This keeps everything consistent and avoids hard-coding numbers.
D_d  = cosmo.angular_diameter_distance(z_d)           # [Mpc]
D_s  = cosmo.angular_diameter_distance(z_s)           # [Mpc]
D_ds = cosmo.angular_diameter_distance_z1z2(z_d, z_s) # [Mpc]

# 3. Einstein radius (arcsec)
#    Roughly the radius of the bright ring for RX J1131.
theta_E_arcsec = 1.84

# 4. Galaxy light / mass shape parameters (elliptical + PA)
#    q and φ chosen to roughly match the Claeskens/Sluse SIE+γ model:
axis_ratio_q = 0.84          # b/a (mass quite round but not perfectly)
position_angle_phi = -73.0   # degrees, measured E of N (approximate)

# 5. External shear (environment)
#    RX J1131 sits in a fairly rich environment; values below are typical
#    of SIE+γ fits (can refine from literature later).
gamma_ext     = 0.11         # shear amplitude (dimensionless)
phi_gamma_deg = -80.0        # shear PA in degrees (E of N, approximate)

# 6. External convergence (LOS mass sheet)
#    H0LiCOW / environment studies suggest κ_ext ~ 0.1 for RX J1131.
kappa_env = 0.10

# 7. Image positions A, B, C, D (arcsec, lens-centred, RX J1131)
# These come from the Witt+ / Claeskens-style astrometry:
# A,B,C,D are quasar images, centred relative to the main lens galaxy G.

image_positions = {
    "A": ( 2.016, -0.610),   # (0.000 - x_G, 0.000 - y_G)
    "B": ( 2.048,  0.578),   # (0.032 - x_G, 1.188 - y_G)
    "C": ( 1.426, -1.730),   # (-0.590 - x_G, -1.120 - y_G)
    "D": (-1.096, 0.274),    # (-3.112 - x_G, 0.884 - y_G)
}

# 8. MGE parameters (elliptical multi-Gaussian for RX J1131)
# Much more centrally concentrated than J1206 — roughly matching a big bulge.
# The absolute scaling is still fixed later by the |∇ψ| = θ_E condition.

MGE_sigmas_arcsec = [
    0.10, 0.20, 0.35, 0.50, 0.75,
    1.00, 1.40, 1.90, 2.60, 3.50
]

MGE_amps = [
    45.0, 30.0, 20.0, 14.0, 10.0,
     6.0,  4.0,  2.5,  1.2,  0.6
]



In [ ]:
# CELL 2 — Build grid & MGE model (does not change)

arcsec_to_rad = (np.pi/180)/3600

x_vals_arcsec = np.linspace(-5, 5, 500)
y_vals_arcsec = np.linspace(-5, 5, 500)
x_grid_arcsec, y_grid_arcsec = np.meshgrid(x_vals_arcsec, y_vals_arcsec)

def elliptical_gaussian(x, y, sigma, amp, q, phi_deg):
    phi = np.deg2rad(phi_deg)
    cosp, sinp = np.cos(phi), np.sin(phi)
    x_rot =  cosp*x + sinp*y
    y_rot = -sinp*x + cosp*y
    r2 = x_rot**2 + (y_rot/q)**2
    return amp*np.exp(-0.5*r2/sigma**2)

def psi_norm(x, y):
    psi = np.zeros_like(x)
    for sigma, amp in zip(MGE_sigmas_arcsec, MGE_amps):
        psi += elliptical_gaussian(
            x, y, sigma, amp,
            axis_ratio_q, position_angle_phi
        )
    return psi

psi_mass_norm = psi_norm(x_grid_arcsec, y_grid_arcsec)

In [ ]:
# CELL 3 — Normalize ψ via |∇ψ| = θ_E (does not change)

# Gradients
dpsi_dy_arcsec, dpsi_dx_arcsec = np.gradient(
    psi_mass_norm,
    y_vals_arcsec,
    x_vals_arcsec
)

# convert to radian deflection
dpsi_dx_rad = dpsi_dx_arcsec / arcsec_to_rad
dpsi_dy_rad = dpsi_dy_arcsec / arcsec_to_rad
alpha_mag = np.sqrt(dpsi_dx_rad**2 + dpsi_dy_rad**2)

# Einstein ring mask
theta_E_rad = theta_E_arcsec * arcsec_to_rad
r_grid = np.sqrt(x_grid_arcsec**2 + y_grid_arcsec**2)
ring_mask = np.abs(r_grid - theta_E_arcsec) <= 0.1

mean_defl_norm = alpha_mag[ring_mask].mean()
A_scale = theta_E_rad / mean_defl_norm

psi_mass_phys = psi_mass_norm * A_scale

In [ ]:
# CELL 4 — Add external shear (does not change)

phi_g = np.deg2rad(phi_gamma_deg)
cos2phi = np.cos(2*phi_g)
sin2phi = np.sin(2*phi_g)

x_rad = x_grid_arcsec * arcsec_to_rad
y_rad = y_grid_arcsec * arcsec_to_rad

psi_shear = 0.5*gamma_ext * (
    (x_rad**2 - y_rad**2)*cos2phi +
    2*x_rad*y_rad*sin2phi
)

psi_total = psi_mass_phys + psi_shear

In [ ]:
# CELL 5 — Evaluate ψ_SFH at image positions and compute Δt (does not change)

def psi_at(theta_x, theta_y):
    return griddata(
        (x_grid_arcsec.ravel(), y_grid_arcsec.ravel()),
        psi_total.ravel(),
        (theta_x, theta_y),
        method='cubic'
    )

# Extract real images
psiA = psi_at(*image_positions["A"])
psiB = psi_at(*image_positions["B"])

delta_psi = abs(psiB - psiA)

delta_t_raw = (delta_psi * time_delay_distance_factor / c) / (24*3600)
delta_t_corr = delta_t_raw / (1 - kappa_env)

print(f"ψ_A = {psiA:.6e}")
print(f"ψ_B = {psiB:.6e}")
print(f"Δψ = {delta_psi:.6e}")
print(f"Raw SFH Δt = {delta_t_raw:.2f} days")
print(f"κ_env-corrected = {delta_t_corr:.2f} days")


ψ_A = 2.468549e-11
ψ_B = 1.903750e-11
Δψ = 5.647991e-12
Raw SFH Δt = 15.63 days
κ_env-corrected = 17.36 days
